# Document Splitting

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [3]:
chunk_size =26
chunk_overlap = 4

In [11]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

NOTE: the length of `text1` below is same length as the `chunk_size` variable.  So, the result is a one element list.

In [5]:
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

NOTE: that `text2` is longer than `chunk_size`.  This give us multiple elements in the resulting list and the second element starts with the last four (because of `chunk_overlap`) of the previous chunk.

In [6]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

Ok, this splits the string but we have an overlap specified as 5, but it looks like 3? (try an even number)

In [7]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [8]:
c_splitter.split_text(text3)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

NOTE: the CharacterTextSplitter splits by `\n` by default.

In [9]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [14]:
text4 = """four score
and seven years ago
our fathers brought to this continent
a new nation
conceived in liberty.
four score
and seven years ago
our fathers brought to this continent
a new nation
conceived in liberty.
four score
and seven years ago
our fathers brought to this continent
a new nation
conceived in liberty.
four score
and seven years ago
our fathers brought to this continent
a new nation
conceived in liberty.
four score
and seven years ago
our fathers brought to this continent
a new nation
conceived in liberty.
"""

In [20]:
c_splitter = CharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=4,
)
c_splitter.split_text(text4)

['four score\nand seven years ago\nour fathers brought to this continent\na new nation\nconceived in liberty.\nfour score\nand seven years ago\nour fathers brought to this continent\na new nation\nconceived in liberty.\nfour score\nand seven years ago\nour fathers brought to this continent\na new nation\nconceived in liberty.\nfour score\nand seven years ago\nour fathers brought to this continent\na new nation\nconceived in liberty.\nfour score\nand seven years ago\nour fathers brought to this continent\na new nation\nconceived in liberty.']

## Recursive splitting details

`RecursiveCharacterTextSplitter` is recommended for generic text. 

In [21]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [22]:
len(some_text)

496

In [23]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [24]:
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [25]:
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

Let's reduce the chunk size a bit and add a period to our separators:

In [26]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [27]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [28]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("machineLearning-Lecture01.pdf")
pages = loader.load()

In [29]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [30]:
docs = text_splitter.split_documents(pages)

In [31]:
len(docs)

77

In [32]:
len(pages)

22

## Token splitting

We can also split on token count explicity, if we want.

This can be useful because LLMs often have context windows designated in tokens.

Tokens are often ~4 characters.

In [33]:
from langchain.text_splitter import TokenTextSplitter

In [34]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [35]:
text1 = "foo bar bazzyfoo"

In [36]:
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [37]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

In [38]:
docs = text_splitter.split_documents(pages)

In [42]:
docs[500]

Document(page_content='problem that you find interesting, and apply lear n', metadata={'source': 'machineLearning-Lecture01.pdf', 'page': 6})

In [40]:
pages[0].metadata

{'source': 'machineLearning-Lecture01.pdf', 'page': 0}

## Context aware splitting

Chunking aims to keep text with common context together.

A text splitting often uses sentences or other delimiters to keep related text together but many documents (such as Markdown) have structure (headers) that can be explicitly used in splitting.

We can use `MarkdownHeaderTextSplitter` to preserve header metadata in our chunks, as show below.

In [43]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [44]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n 
## Chapter 2\n\n \
Hi this is Molly"""

In [50]:
headers_to_split_on = [
    ("#", "Big 1"),
    ("##", "medium 2"),
    ("###", "not so big 3"),
]

In [51]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [52]:
md_header_splits[0]

Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Big 1': 'Title', 'medium 2': 'Chapter 1'})

In [53]:
md_header_splits[1]

Document(page_content='Hi this is Lance', metadata={'Big 1': 'Title', 'medium 2': 'Chapter 1', 'not so big 3': 'Section'})

In [54]:
md_header_splits[2]

Document(page_content='Hi this is Molly', metadata={'Big 1': 'Title', 'medium 2': 'Chapter 2'})